## 0. Colab Code

## 1. Preparing the imports

In [1]:
import torch
import torchaudio
import torch.nn.functional as F

import os
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from IPython.display import Audio
import time

## 0.5. Creating the Dataset

## 2. HuBERT TEST

In [2]:
from transformers import HubertModel,AutoProcessor,AutoFeatureExtractor,Wav2Vec2Processor,HubertForCTC
import joblib
import torch.nn as nn
from hubertKM import SemanticTokenizer, visualizeEmbeddings

c:\Users\fabri\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Importing the pretrained models for HuBERT and KMeans

In [3]:
# A semantic tokenizer
# Input: (w2vCheckpointPath, kmeansCheckpointPath)
# Output: (semanticTokens, normalizedEmbeddings)
w2vBERT = SemanticTokenizer("facebook/hubert-base-ls960","./hubertKM/hubert_base_ls960_L9_km500.bin")  

Some weights of the model checkpoint at facebook/hubert-base-ls960 were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertModel were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

#### Computing semantic tokens

In [4]:
#semanticTokens, embeddings

#### Visualizing embeddings

In [5]:
# Create some plots using PCA and t-SNE

# visualizeEmbeddings(embeddings, semanticTokens)

#### Check with another implementation (OPTIONAL)

## 3. Creating the Semantic Transformer

In [6]:
from SoundStream import soundstream_16khz, audio_to_tokens, tokens_to_audio, encode_audio, decode_audio

In [7]:
soundStream = soundstream_16khz()

## TEST

In [8]:
from data import storeTokens,  TokensDataset, store_from_librilight

tokenPath = "out"
tokenFile = "out.csv"
audioPath = "data_cut"


In [16]:
#fileCount = store_from_librilight(tokenPath, tokenFile, w2vBERT, soundStream, fileCountCheckpoint = 10)

ValueError: too many values to unpack (expected 2)

In [10]:
#tokenDataset = TokensDataset(tokenPath, tokenFile, requiredDuration = 30, includeSemanticTokens = True, includeCoarseTokens = True, includeFineTokens = True) 
#semanticDataset = TokensDataset(tokenPath, tokenFile, requiredDuration = 30, includeSemanticTokens = True)
coarseDataset = TokensDataset(tokenPath, tokenFile, requiredDuration = 10, includeCoarseTokens = True)
#fineDataset = TokensDataset(tokenPath, tokenFile, requiredDuration = 30, includeFineTokens = True)

In [11]:
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from TransformerModel import Decoder
import pytorch_lightning as pl

In [13]:
#dataset = semanticDataset
train_dataset = coarseDataset

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=6)


In [14]:

#Change vocab_size and seq_len to initiate, k should be d_model/num_heads
model = Decoder( d_model=256, num_layers = 3, num_heads=4, dim_feedforward=1024, dropout=0.1, k=64, vocab_size=10000, seq_len=1503)


In [ ]:
checkpoint = torch.load("checkpoints\checkpoint_epoch_10.pt")
model.load_state_dict(checkpoint['model_state_dict'])
model.load_state_dict(checkpoint['model_state_dict'])
model.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
C:\Users\fabri\AppData\Local\Temp\ipykernel_17196\3180745041.py:3: SyntaxWarning: invalid escape sequence '\c'
  checkpoint = torch.load("checkpoints\checkpoint_epoch_10.pt")


In [ ]:
model.train(train_dataset)

In [ ]:
#model.eval()
x_input = coarseDataset.__getitem__(0)

print(x_input)
with torch.no_grad():
	predictions = model(x_input[0])

(tensor([[  85, 1245, 2766,  ...,  476, 1325, 2570]]), tensor([[1245, 2766,  556,  ..., 1325, 2570, -100]]))


In [ ]:
pred = torch.argmax(predictions, dim=-1)
for elem in pred[0]:
	print(elem)

tensor(1720)
tensor(2483)
tensor(822)
tensor(1241)
tensor(2317)
tensor(822)
tensor(1241)
tensor(822)
tensor(514)
tensor(1076)
tensor(2317)
tensor(773)
tensor(822)
tensor(514)
tensor(773)
tensor(1989)
tensor(514)
tensor(514)
tensor(1241)
tensor(822)
tensor(514)
tensor(1241)
tensor(822)
tensor(822)
tensor(1720)
tensor(822)
tensor(773)
tensor(822)
tensor(822)
tensor(919)
tensor(822)
tensor(822)
tensor(822)
tensor(822)
tensor(436)
tensor(822)
tensor(822)
tensor(822)
tensor(773)
tensor(514)
tensor(507)
tensor(822)
tensor(822)
tensor(787)
tensor(514)
tensor(822)
tensor(2317)
tensor(514)
tensor(787)
tensor(822)
tensor(787)
tensor(436)
tensor(514)
tensor(787)
tensor(514)
tensor(822)
tensor(822)
tensor(1241)
tensor(773)
tensor(773)
tensor(514)
tensor(2031)
tensor(822)
tensor(1989)
tensor(822)
tensor(822)
tensor(514)
tensor(2317)
tensor(787)
tensor(227)
tensor(822)
tensor(514)
tensor(514)
tensor(436)
tensor(2668)
tensor(1076)
tensor(822)
tensor(822)
tensor(822)
tensor(2317)
tensor(227)
tensor(77

In [ ]:
fineDataset = TokensDataset(tokenPath, tokenFile, requiredDuration = 10, includeFineTokens = True)

In [ ]:
y_input = fineDataset.__getitem__(0)[0]

y_input = y_input.squeeze(0)
pred_new = pred.squeeze(0)
#waveform = tokens_to_audio(pred_new, y_input, soundStream)
y_input.shape

torch.Size([2505])